In [64]:
# #This  program is created by rakesh from crddmirror system on Tue 01 Dec 2020 11:22:29 PM IST.
#License: Attribution-NonCommercial-ShareAlike 4.0 International. For feedback, please email at rakesh4osdd@gmail.com
# check for updated code jupyter file name 'virhostnet' in MITAB 2.5 format
import csv
import re
import fileinput

In [65]:
! grep ';'  9606 > 9606_virusmentha.txt  # remove unwanted blank line
# define input and output file with taxid mapping file
input1='9606_virusmentha.txt'
output1='hpi-'+input1
both2='both-'+input1
pathogen3='pathogen-'+input1
mapping_taxid='mapping-203_pathogen_taxid_virusmentha.txt'

In [66]:
# set output file for writing
#fieldnames = [ 'SN', 'protein1', 'protein2', 'host', 'partner']
fieldnames = [ 'host_protein1', 'pathogen_protein2', 'host', 'pathogen']
csvfile1=open(output1, 'w', newline='')
#fieldnames = ['SN', 'protein1', 'protein2', 'host', 'partner']
output = csv.DictWriter(csvfile1,fieldnames=fieldnames, delimiter='\t')
output.writeheader()

csvfile2=open(both2, 'w', newline='')
#fieldnames = ['SN', 'protein1', 'protein2', 'host', 'partner']
both = csv.DictWriter(csvfile2,fieldnames=fieldnames, delimiter='\t')
both.writeheader()

csvfile3=open(pathogen3, 'w', newline='')
#fieldnames = ['SN', 'protein1', 'protein2', 'host', 'partner']
path = csv.DictWriter(csvfile3,fieldnames=fieldnames, delimiter='\t')
path.writeheader()
#path.writerow({'SN': '1', 'protein1': 'protein1', 'protein2': 'protein2' , 'host': 'host', 'partner': 'partner2'})

47

In [67]:
def update_taxid(path_taxid):
    with fileinput.input(mapping_taxid) as f:
        for line in f:
            #print (line[:-2])
            #for line in fileinput.input(files='map2', inplace=False, mode='r'):
            taxid=line.split('\t')
            #print(taxid[0] ,taxid[1][:-2])
            #line = re.sub( taxid[0] ,taxid[1], line.rstrip())
            if re.search( rf'\b{taxid[0]}\b', path_taxid):
                new_taxid = re.sub( rf'\b{taxid[0]}\b' ,taxid[1], path_taxid)
                return str(new_taxid[:-2])
    #fileinput.nextfile()
    #fileinput.close()
#update_taxid('taxid:694009')

In [68]:
# code for filter csv file| c1,5,3,7 |Protein A;Gene A;Taxon A;Family A;Protein B;Gene B;Taxon B;Family B;Score;PMID
with open(input1) as csvfile:
    hpi = csv.reader(csvfile, delimiter=';')
    next(hpi, None)  # skip the headers
    for i, row in enumerate(hpi):
        if i < 3:
            print (i)
        if i >=0:
            #print(row)
            protein1=row[0]
            protein2=row[4]
            # search for homo sapiens 9606
            pattern1=r'\b9606\b.*'
            # search for pair
            pattern2= r"\b9606\b[^:].*\b9606\b.*"
            #print(row[2]+row[3])
            string=row[2]+'\t'+row[6]
            host=row[2]
            pathogen=row[6]
            if re.search(pattern2, string):
                partner2=pathogen
                #both.writerow({'SN': i, 'host_protein1': protein1, 'pathogen_protein2': protein2 , 'host': host, 'pathogen': partner2})
                both.writerow({'host_protein1': protein1, 'pathogen_protein2': protein2 , 'host': host, 'pathogen': partner2})
            elif re.search(pattern1, host):
                # for homogenious hame of host
                host='taxid:9606(Homo sapiens)'
                partner2= update_taxid(pathogen)
                #output.writerow({'SN': i, 'host_protein1': protein1, 'pathogen_protein2': protein2 , 'host': host, 'pathogen': partner2})
                output.writerow({'host_protein1': protein1, 'pathogen_protein2': protein2 , 'host': host, 'pathogen': partner2}) 
            elif re.search(pattern1, pathogen):
                #print(row[0], row[1], row[9],row[10])
                protein1=row[4] # interchange of protein
                protein2=row[0]
                # for homogenious hame of host
                host='taxid:9606(Homo sapiens)'
                partner2=update_taxid(row[2])
                #output.writerow({'SN': i, 'host_protein1': protein1, 'pathogen_protein2': protein2 , 'host': host, 'pathogen': partner2})
                output.writerow({'host_protein1': protein1, 'pathogen_protein2': protein2 , 'host': host, 'pathogen': partner2})
            else:
                partner2=pathogen
                path.writerow({'host_protein1': row[0] , 'pathogen_protein2': row[4] , 'host': row[2], 'pathogen': row[6]})
                #print("No match",row)

csvfile1.flush()
csvfile2.flush()
csvfile3.flush()

0
1
2


In [70]:
# check output files and stats
! head -n2 {output1} {both2} {pathogen3} ; tail -n2 {output1} {both2} {pathogen3}
! wc -l {output1} {both2} {pathogen3} ; wc -l {input1}
#! rm {output1} {both2} {pathogen3}
! tail -n +2 hpi-{input1} |cut -f1,2|sort -u|wc -l ; 
! tail -n +2 {input1} |cut -f1,2|sort -u|wc -l

==> hpi-9606_virusmentha.txt <==
host_protein1	pathogen_protein2	host	pathogen
Q99075	P00587	taxid:9606(Homo sapiens)	taxid:10703(Corynephage beta)

==> both-9606_virusmentha.txt <==
host_protein1	pathogen_protein2	host	pathogen

==> pathogen-9606_virusmentha.txt <==
host_protein1	pathogen_protein2	host	pathogen
==> hpi-9606_virusmentha.txt <==
Q9HC16	P69720	taxid:9606(Homo sapiens)	taxid:11676(Human immunodeficiency virus 1)
Q9HC16	P69721	taxid:9606(Homo sapiens)	taxid:11676(Human immunodeficiency virus 1)

==> both-9606_virusmentha.txt <==
host_protein1	pathogen_protein2	host	pathogen

==> pathogen-9606_virusmentha.txt <==
host_protein1	pathogen_protein2	host	pathogen
 10693 hpi-9606_virusmentha.txt
     1 both-9606_virusmentha.txt
     1 pathogen-9606_virusmentha.txt
 10695 total
10693 9606_virusmentha.txt
10692
10692


In [71]:
csvfile1.close()
csvfile2.close()
csvfile3.close()

In [72]:
# check for Mt Protein exist in the hpi-{input1} file using Mt_Proteins_7432_working_uniprot_ids.txt and mt-protein-uniprot.txt
! head -n1 hpi-{input1} > mnDNA_hpi-{input1}
! grep -a -w -f ../Mt_Proteins_7432_working_uniprot_ids.txt hpi-{input1} >> mnDNA_hpi-{input1}
! wc -l mnDNA_hpi-{input1}; wc -l hpi-{input1}
! head -n2 mnDNA_hpi-{input1} hpi-{input1}

5028 mnDNA_hpi-9606_virusmentha.txt
10693 hpi-9606_virusmentha.txt
==> mnDNA_hpi-9606_virusmentha.txt <==
host_protein1	pathogen_protein2	host	pathogen
Q96EY1	P03129	taxid:9606(Homo sapiens)	taxid:333760(Human papillomavirus type 16)

==> hpi-9606_virusmentha.txt <==
host_protein1	pathogen_protein2	host	pathogen
Q99075	P00587	taxid:9606(Homo sapiens)	taxid:10703(Corynephage beta)


In [73]:
column4file='mnDNA_hpi-'+input1
! wc -l {column4file}
for i in range(4):
    header = ! head -n1 {column4file} |cut -f{i+1}
    count = ! tail -n +2 {column4file} |cut -f{i+1}  |sort -u|wc -l
    print (header[0], count[0])

5028 mnDNA_hpi-9606_virusmentha.txt
host_protein1 1348
pathogen_protein2 402
host 1
pathogen 158


In [74]:
# split file into mDNA and nDNA list
! head -n1 mnDNA_hpi-{input1} > mDNA_hpi-{input1}
! grep -a -w -f ../mt-protein-uniprot.txt mnDNA_hpi-{input1} >> mDNA_hpi-{input1}
! grep -a -v -w -f ../mt-protein-uniprot.txt mnDNA_hpi-{input1} > nDNA_hpi-{input1}
! head -n3 mDNA_hpi-{input1} nDNA_hpi-{input1} mnDNA_hpi-{input1}
! wc -l mDNA_hpi-{input1} nDNA_hpi-{input1} ; wc -l mnDNA_hpi-{input1}

==> mDNA_hpi-9606_virusmentha.txt <==
host_protein1	pathogen_protein2	host	pathogen
P00846	P0C739	taxid:9606(Homo sapiens)	taxid:10377(Human herpesvirus 4 strain B95-8)
P00403	P69901	taxid:9606(Homo sapiens)	taxid:10580(human papillomavirus 11)

==> nDNA_hpi-9606_virusmentha.txt <==
host_protein1	pathogen_protein2	host	pathogen
Q96EY1	P03129	taxid:9606(Homo sapiens)	taxid:333760(Human papillomavirus type 16)
Q99683	P69720	taxid:9606(Homo sapiens)	taxid:11676(Human immunodeficiency virus 1)

==> mnDNA_hpi-9606_virusmentha.txt <==
host_protein1	pathogen_protein2	host	pathogen
Q96EY1	P03129	taxid:9606(Homo sapiens)	taxid:333760(Human papillomavirus type 16)
Q99683	P69720	taxid:9606(Homo sapiens)	taxid:11676(Human immunodeficiency virus 1)
    12 mDNA_hpi-9606_virusmentha.txt
  5017 nDNA_hpi-9606_virusmentha.txt
  5029 total
5028 mnDNA_hpi-9606_virusmentha.txt


In [75]:
files= ! ls nDNA_hpi-{input1} mDNA_hpi-{input1}
for file in files:
    column4file=file
    ! wc -l {column4file}
    for i in range(4):
        header = ! head -n1 {column4file} |cut -f{i+1}
        count = ! tail -n +2 {column4file} |cut -f{i+1}  |sort -u|wc -l
        print (header[0], count[0])

12 mDNA_hpi-9606_virusmentha.txt
host_protein1 3
pathogen_protein2 7
host 1
pathogen 6
5017 nDNA_hpi-9606_virusmentha.txt
host_protein1 1345
pathogen_protein2 402
host 1
pathogen 158


In [76]:
"""def update_taxid(path_taxid):
    return path_taxid"""
# to create mapping file
"""! tail -n +2 mnDNA_hpi-{input1}|cut -f4|sort -un > 158_pathogen_taxid_virusmentha.txt
! wc -l 158_pathogen_taxid_virusmentha.txt; head -n3 158_pathogen_taxid_virusmentha.txt"""

'! tail -n +2 mnDNA_hpi-{input1}|cut -f4|sort -un > 158_pathogen_taxid_virusmentha.txt\n! wc -l 158_pathogen_taxid_virusmentha.txt; head -n3 158_pathogen_taxid_virusmentha.txt'

123 4
456 456 123456


In [77]:
! head -n3 hpi-9606_virusmentha.txt

In [78]:
column4file='hpi-9606_virusmentha.txt'
! wc -l {column4file}
for i in range(4):
    header = ! head -n1 {column4file} |cut -f{i+1}
    count = ! tail -n +2 {column4file} |cut -f{i+1}  |sort -u|wc -l
    print (header[0], count[0])

10693 hpi-9606_virusmentha.txt
host_protein1 3543
pathogen_protein2 585
host 1
pathogen 203
